In [ ]:
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [1]:
from nltk.tokenize import word_tokenize
import nltk
def load_data(file_path):
    #file_path에 있는 데이터를 읽어 옴
    
    train = []
    
    count = 0
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            if count == 500: break     # 500개만 가져옴
            line = line.strip()
            id, doc, label = line.split('\t')
            if label == '1': label = 'pos'
            elif label == '0': label = 'neg'
            train.append((doc,label))
            count += 1
            
    return train

In [2]:
train = load_data('ratings_train.txt')
print(train[:5])

[('document', 'label'), ('아 더빙.. 진짜 짜증나네요 목소리', 'neg'), ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'pos'), ('너무재밓었다그래서보는것을추천한다', 'neg'), ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'neg')]


In [3]:
train = train[1:]
print(train[:5])

[('아 더빙.. 진짜 짜증나네요 목소리', 'neg'), ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'pos'), ('너무재밓었다그래서보는것을추천한다', 'neg'), ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'neg'), ('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'pos')]


## 작성한 내용 함수로 만들기

In [8]:
from konlpy.tag import Okt
okt = Okt()

def pos_tokenize(raw_sent):
    pos_sent = []
    
    #raw_sent: 사과가 좋아
    sent = okt.pos(raw_sent, norm=True, stem=True)
    #sent: [('사과', 'Noun'),('가','Josa'),('좋다','Adjective')]
    
    for tup in sent: 
        word, tag = tup[0], tup[1]    # tup: ('사과', 'Noun')
        word_tag = word + '/' + tag  # word_tag: '사과/Noun'
        pos_sent.append(word_tag)
        
    return ' '.join(pos_sent)

def make_word_dict(train, use_morph=False):
    all_words = set()

    for tup in train:
        sent, label = tup[0], tup[1]  #ex) sent = 'l like you', label = 'pos'
        if use_morph: sent = pos_tokenize(sent)
        words = word_tokenize(sent)
        for word in words:
            all_words.add(word)
    return all_words
def make_train_features(train, all_words, use_morph=False):
    train_features = []

    for tup in train:
        sent, label = tup[0], tup[1]
        if use_morph: sent = pos_tokenize(sent)
        words = word_tokenize(sent)
        tmp = {set_word: (set_word in words) for set_word in all_words} # set_word가 words에 있는지 여부 판별
        sent_tup = (tmp, label)
        train_features.append(sent_tup)
            
    return train_features

## 테스트하기 1

In [9]:
use_morph = True
all_words = make_word_dict(train, use_morph)
print('단어 집합 개수: ', len(all_words))

train_features = make_train_features(train, all_words, use_morph)

단어 집합 개수:  2322


In [10]:
classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features()
## classifier.show_most_informative_features(n=5)

Most Informative Features
                       ; = True              neg : pos    =      8.2 : 1.0
          재미없다/Adjective = True              neg : pos    =      8.2 : 1.0
                주인공/Noun = True              neg : pos    =      7.5 : 1.0
                 최고/Noun = True              pos : neg    =      6.9 : 1.0
                  뭐/Noun = True              neg : pos    =      6.8 : 1.0
           재밌다/Adjective = True              pos : neg    =      6.5 : 1.0
                 내용/Noun = True              neg : pos    =      6.2 : 1.0
       ㅡㅡ/KoreanParticle = True              neg : pos    =      6.1 : 1.0
                스토리/Noun = True              neg : pos    =      6.1 : 1.0
                 다시/Noun = True              pos : neg    =      5.9 : 1.0


In [11]:
test_sent = '보다가 중간에 나왔습니다.'
if use_morph: test_sent = pos_tokenize(test_sent)
words = word_tokenize(test_sent)
test_feature = {set_word: (set_word in words) for set_word in all_words}

In [12]:
classifier.classify(test_feature)

'pos'

## 테스트하기 2

In [13]:
use_morph = False
all_words = make_word_dict(train, use_morph)
print('단어 집합 개수: ', len(all_words))

train_features = make_train_features(train, all_words, use_morph)

단어 집합 개수:  3046


In [14]:
classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features()

Most Informative Features
                       ; = True              neg : pos    =      8.2 : 1.0
                       그 = True              pos : neg    =      4.7 : 1.0
                       수 = True              pos : neg    =      4.7 : 1.0
                      ㅡㅡ = True              neg : pos    =      4.6 : 1.0
                       ! = True              pos : neg    =      4.0 : 1.0
                    스토리도 = True              neg : pos    =      3.9 : 1.0
                      이거 = True              neg : pos    =      3.9 : 1.0
                     하나도 = True              neg : pos    =      3.9 : 1.0
                      없다 = True              neg : pos    =      3.6 : 1.0
                       ? = True              neg : pos    =      3.6 : 1.0


In [16]:
test_sent = '보다가 중간에 나왔습니다.'
if use_morph: test_sent = pos_tokenize(test_sent)
words = word_tokenize(test_sent)
test_feature = {set_word: (set_word in words) for set_word in all_words}

classifier.classify(test_feature)

'neg'

# 나이브 베이즈 분류기

In [43]:
import math

class MyNaiveBayesClassifier:
    
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []
        self.use_morph = use_morph
        
        if self.use_morph:
            from konlpy.tag import Okt
            self.okt = Okt()   

        
    def load_data(self, file_path):
        # file_path에 있는 데이터를 읽어 옴
        docs = []
        labels = []
        count = 0
        with open(file_path, 'r',encoding='utf-8')as f:
            for line in f.readlines():
                if count == 500: break
                line = line.strip()
                id, doc, label = line.split('\t')
                docs.append(doc)
                if label =='1': label = 'pos'
                elif label == '0': label = 'neg'
                labels.append(label)
                count += 1
                
        return docs[1:], labels[1:]
    def tokenize(self, sentence):
        if self.use_morph:
            pos_sent = []

            #raw_sent: 사과가 좋아
            sent = okt.pos(raw_sent, norm=True, stem=True)
            #sent: [('사과', 'Noun'),('가','Josa'),('좋다','Adjective')]

            for tup in sent: 
                word, tag = tup[0], tup[1]    # tup: ('사과', 'Noun')
                word_tag = word + '/' + tag  # word_tag: '사과/Noun'
                pos_sent.append(word_tag)

            sentence =  ' '.join(pos_sent)
            
        return sentence.split()
    def count_words(self, docs, labels): 
        # 단어사전을 만들고, 각 단어의 긍/부정 문서 등장 횟수 세기
        
        count_dict = dict()
        for doc, label in zip(docs, labels):
            print(doc)
            for word in self.tokenize(doc):
                if word not in count_dict:
                    count_dict[word] = {'pos':0, 'neg':0}
                
                count_dict[word][label] += 1
        print('num of words...', len(count_dict))
        return count_dict
    def word_prob(self, count_dict, pos_class_num, neg_class_num, k):
        # (단어, p(단어|긍정), p(단어|부정))의 튜플 형태로 만들어주어 리스트에 추가
        worb_prob_list = []
        
        for key in count_dict:
            pos_word_num = count_dict[key]['pos']
            neg_word_num = count_dict[key]['neg']
            
            pos_class_prob = (pos_word_num + k) / (pos_class_num + 2*k)
            neg_class_prob = (neg_word_num + k) / (neg_class_num + 2*k)
            
            tup = (key, pos_class_prob, neg_class_prob)
            word_prob_list.append(tup)
            
        return word_prob_list
    def class_prob(self, word_prob_list, test_sentence, use_unseen=False):
        # p(긍정|문서), p(부정|문서) 계산
        
        test_words = self.tokenize(test_sentence)
        
        sent_log_pos_class_prob, sent_log_neg_class_prob = 0.0, 0.0
        
        for word, word_pos_class_prob, word_neg_class_prob in word_prob_list:
            if word in test_words:
                sent_log_pos_class_prob = sent_log_pos_class_prob + math.log(word_pos_class_prob)
                sent_log_neg_class_prob = sent_log_neg_class_prob + math.log(word_neg_class_prob)
            else:
                if use_unseen:
                    sent_log_pos_class_prob = sent_log_pos_class_prob + math.log(1.0-word_pos_class_prob)
                    sent_log_neg_class_prob = sent_log_neg_class_prob + math.log(1.0-word_neg_class_prob)
        
        sent_pos_class_prob = math.exp(sent_log_pos_class_prob)
        sent_neg_class_prob = math.exp(sent_log_neg_class_prob)
        
        pos_class_prob = sent_pos_class_prob/(sent_pos_class_prob+sent_neg_class_prob)
        neg_class_prob = sent_neg_class_prob/(sent_pos_class_prob+sent_neg_class_prob)
        
        return pos_class_prob, neg_class_prob
        
    def train(self, train_file_path):
        # load_date, count_words, word_prob 계산
        train_docs, train_labels = self.load_data(train_file_path)
        
        word_count_dict = self.count_words(train_docs, train_labels)
        
        pos_class_num = len([label for label in train_labels if label == 'pos'])
        neg_class_num = len([label for label in train_labels if label == 'neg'])
        
        self.word_probs = self.word_prob(word_count_dict, pos_class_num, neg_class_num, self.k)
    def classify(self, doc, use_unseen=False):
        # class_prob 계산
        pos_class_prob, neg_class_prob = self.class_prob(self.word_probs, doc, use_unseen)
        
        if pos_class_prob > neg_class_prob:
            print('pos', pos_class_prob)
        else:
            print('neg', neg_class_prob)


In [44]:
classifier = MyNaiveBayesClassifier()

In [45]:
classifier.train('ratings_train.txt')

아 더빙.. 진짜 짜증나네요 목소리
아 neg
더빙.. neg
진짜 neg
짜증나네요 neg
목소리 neg
흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
흠...포스터보고 pos
초딩영화줄....오버연기조차 pos
가볍지 pos
않구나 pos
너무재밓었다그래서보는것을추천한다
너무재밓었다그래서보는것을추천한다 neg
교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
교도소 neg
이야기구먼 neg
..솔직히 neg
재미는 neg
없다..평점 neg
조정 neg
사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
사이몬페그의 pos
익살스런 pos
연기가 pos
돋보였던 pos
영화!스파이더맨에서 pos
늙어보이기만 pos
했던 pos
커스틴 pos
던스트가 pos
너무나도 pos
이뻐보였다 pos
막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
막 neg
걸음마 neg
뗀 neg
3세부터 neg
초등학교 neg
1학년생인 neg
8살용영화.ㅋㅋㅋ...별반개도 neg
아까움. neg
원작의 긴장감을 제대로 살려내지못했다.
원작의 neg
긴장감을 neg
제대로 neg
살려내지못했다. neg
별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네
별 neg
반개도 neg
아깝다 neg
욕나온다 neg
이응경 neg
길용우 neg
연기생활이몇년인지..정말 neg
발로해도 neg
그것보단 neg
낫겟다 neg
납치.감금만반복반복..이드라마는 neg
가족도없다 neg
연기못하는사람만모엿네 neg
액션이 없는데도 재미 있는 몇안되는 영화
액션이 pos
없는데도 pos
재미 pos
있는 pos
몇안되는 pos
영화 pos
왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?
왜케 pos
평점이 pos
낮은건데? pos
꽤 pos
볼만한데.. 

코미디의한장면 neg
같음(웃음) neg
"""영화 끝나갈때쯤에 멍하다가 다보고나면 한마디 나올거임 """"ㅈ같다."""""""
"""영화 pos
끝나갈때쯤에 pos
멍하다가 pos
다보고나면 pos
한마디 pos
나올거임 pos
""""ㅈ같다.""""""" pos
공유 존잘!!!ㅎㅎㅎ
공유 pos
존잘!!!ㅎㅎㅎ pos
상쾌발랄한 영화다. 말하기 껄끄런 성이란 소재를 유쾌하게 해설했다.
상쾌발랄한 pos
영화다. pos
말하기 pos
껄끄런 pos
성이란 pos
소재를 pos
유쾌하게 pos
해설했다. pos
소파에 죽 치고 앉아 지켜 볼 이유가 없는 작품.
소파에 neg
죽 neg
치고 neg
앉아 neg
지켜 neg
볼 neg
이유가 neg
없는 neg
작품. neg
로큰롤!!!!!!!!!!!!!!
로큰롤!!!!!!!!!!!!!! pos
주된 타겟이 어린이니 일반적인 논리가 통하지 않는 건 알겠다. 하지만 게임은 흥미롭지 않고, 요원이라는 주인공이 너무 무능력해서 별로 재미없다. CG 배경도 거슬린다.
주된 neg
타겟이 neg
어린이니 neg
일반적인 neg
논리가 neg
통하지 neg
않는 neg
건 neg
알겠다. neg
하지만 neg
게임은 neg
흥미롭지 neg
않고, neg
요원이라는 neg
주인공이 neg
너무 neg
무능력해서 neg
별로 neg
재미없다. neg
CG neg
배경도 neg
거슬린다. neg
뮤지컬 영화인데 사운드 녹음 엉망, 남주는 춤도 못추고, 내용은 뻔할뻔, 주인공들 목소리도 너무 안어울리고 어제 CGV에서 뛰어 나가려다 참았습니다.진심 말리고 싶습니다. 영국의 저예산 DVD용 영화뮤지컬영화 아닙니다. 맘마미아 1/10도 안됨
뮤지컬 neg
영화인데 neg
사운드 neg
녹음 neg
엉망, neg
남주는 neg
춤도 neg
못추고, neg
내용은 neg
뻔할뻔, neg
주인공들 neg
목소리도 neg
너무 neg
안어울리고 neg
어제 neg
CGV에서 neg
뛰어 neg
나가려다 neg
참았습니다.진심 neg
말리고 

안개낀 pos
워터루 pos
다리와 pos
마스코트. pos
곤사토시 감독... 2010년 안타깝게 돌아가셔서 가슴이 먹먹합니다.. 정말 천재적인 감독인데 암으로 가시다니.. ㅠㅠ 이제와서 다시봐도 모든 작품이 대작...
곤사토시 pos
감독... pos
2010년 pos
안타깝게 pos
돌아가셔서 pos
가슴이 pos
먹먹합니다.. pos
정말 pos
천재적인 pos
감독인데 pos
암으로 pos
가시다니.. pos
ㅠㅠ pos
이제와서 pos
다시봐도 pos
모든 pos
작품이 pos
대작... pos
많은생각을 하게 됐습니다. 예뼈지고 싶은 맘은 있었지만 과하면 독이겠네요. ^^
많은생각을 pos
하게 pos
됐습니다. pos
예뼈지고 pos
싶은 pos
맘은 pos
있었지만 pos
과하면 pos
독이겠네요. pos
^^ pos
아 츠무구만 없어지면 별 5개줄게
아 neg
츠무구만 neg
없어지면 neg
별 neg
5개줄게 neg
어릴땐 조폭영화로 알다가 나이가들수록 이게 인생이구나 하고 뭉클해지는 영화.세상을 살아가면 갈수록 와닿는게 많아지네요...세상은 비정하지만 비정함마져도 따뜻해지는게 친구라고...
어릴땐 pos
조폭영화로 pos
알다가 pos
나이가들수록 pos
이게 pos
인생이구나 pos
하고 pos
뭉클해지는 pos
영화.세상을 pos
살아가면 pos
갈수록 pos
와닿는게 pos
많아지네요...세상은 pos
비정하지만 pos
비정함마져도 pos
따뜻해지는게 pos
친구라고... pos
영상이 너무나도 멋지다.
영상이 pos
너무나도 pos
멋지다. pos
비디오가 있어서 봤는데 1997년도인줄모를정도로 잘만들었습니다 긴장감도있고
비디오가 pos
있어서 pos
봤는데 pos
1997년도인줄모를정도로 pos
잘만들었습니다 pos
긴장감도있고 pos
처ㅝ주
처ㅝ주 pos
2009년에 만들어진것치곤 재밌음 영화가 길어서 좀 다듬었으면 ..소재도 좋고 몰입감도 기대이상 매끄럽지 못한 부분도 있지만 연기는 둘 다 잘한듯
2009년에 pos
만들어진것

1편 neg
보다 neg
못한 neg
구성등.. neg
2편은 neg
안나오는게 neg
나았다..망작..배우들이 neg
영화보는 neg
눈이 neg
없어서 neg
안타깝다. neg
ㅋㅋㅋㅋㅋㅋㅋㅋ 반도 안되는 영화
ㅋㅋㅋㅋㅋㅋㅋㅋ neg
반도 neg
안되는 neg
영화 neg
이 영화가 평점이 높은 이유를 모르겠어..ㅡ.ㅡ::
이 neg
영화가 neg
평점이 neg
높은 neg
이유를 neg
모르겠어..ㅡ.ㅡ:: neg
평생 기억할만한 영화,정상적인 소재는 아니지만
평생 pos
기억할만한 pos
영화,정상적인 pos
소재는 pos
아니지만 pos
요즘 상황 보고 이 영화가 생각났다.
요즘 pos
상황 pos
보고 pos
이 pos
영화가 pos
생각났다. pos
많을 것을 생각하게 만드는 영화입니다.마지막에 사람들이 짐승으로 보이고 아수라가 사람같아 보였습니다.
많을 pos
것을 pos
생각하게 pos
만드는 pos
영화입니다.마지막에 pos
사람들이 pos
짐승으로 pos
보이고 pos
아수라가 pos
사람같아 pos
보였습니다. pos
진짜 재미 없는 영화 공통점....스포츠영화,군대영화,자막 나오는 영화
진짜 neg
재미 neg
없는 neg
영화 neg
공통점....스포츠영화,군대영화,자막 neg
나오는 neg
영화 neg
로버트 드니로의 광기의 복수 연기를 만끽할수 있는 수작 서스펜스 스릴러물
로버트 pos
드니로의 pos
광기의 pos
복수 pos
연기를 pos
만끽할수 pos
있는 pos
수작 pos
서스펜스 pos
스릴러물 pos
감동감동감동의 도가니탕
감동감동감동의 pos
도가니탕 pos
세계 어디서나 정치 경제 문화 사회 전반에 걸쳐 변화가 절실한 상황이지만 변화를 가져올 방법이 없다는게 함정인 것 같다...
세계 pos
어디서나 pos
정치 pos
경제 pos
문화 pos
사회 pos
전반에 pos
걸쳐 pos
변화가 pos
절실한 pos
상황이지만 pos
변화를 pos
가져올 pos
방법이 pos
없다는게 pos
함정인 pos
것 pos
같

NameError: name 'word_prob_list' is not defined

In [46]:
classifier.classify('최고의 영화예요!')

neg 0.5


In [47]:
test_docs, test_labels = classifier.load_data('ratings_train.txt')